<a href="https://colab.research.google.com/github/aashitadutta/FinancialNarrativeSummarization/blob/main/tf_idf_summarization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:

### importing the necessary libraries
import os
from nltk.corpus import stopwords
import numpy as np
import pandas
import nltk
import re
from nltk import sent_tokenize

In [ ]:
inp_data="/content/drive/MyDrive/lstm-input/"
out_data="/content/drive/MyDrive/lstm-summ/"

datasets={"text":inp_data,"summ":out_data}

data_categories=["training","validation","test"]

data={"articles":[],"summaries":[]}

In [ ]:
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
!pip install Rouge

In [ ]:
!pip install rouge_score

In [ ]:
import math

In [ ]:
def parsetext(dire,category,filename):
    with open("%s/%s"%(dire,filename),'r',encoding="Latin-1") as readin:
        print("file read successfully")
        text=readin.read()
    return text.lower()


def load_data(dire,category):
    for (root,dirs,files) in os.walk(dire):
         filenames=sorted(files)
    return filenames


def cleantext(text):
    text=re.sub(r"what's","what is ",text)
    text=re.sub(r"it's","it is ",text)
    text=re.sub(r"\'ve"," have ",text)
    text=re.sub(r"i'm","i am ",text)
    text=re.sub(r"\'re"," are ",text)
    text=re.sub(r"n't"," not ",text)
    text=re.sub(r"\'d"," would ",text)
    text=re.sub(r"\'s","s",text)
    text=re.sub(r"\'ll"," will ",text)
    text=re.sub(r"can't"," cannot ",text)
    text=re.sub(r" e g "," eg ",text)
    text=re.sub(r"e-mail","email",text)
    text=re.sub(r"9\\/11"," 911 ",text)
    text=re.sub(r" u.s"," american ",text)
    text=re.sub(r" u.n"," united nations ",text)
    text=re.sub(r"\n"," ",text)
    text=re.sub(r":"," ",text)
    text=re.sub(r"-"," ",text)
    text=re.sub(r"\_"," ",text)
    text=re.sub(r"\d+"," ",text)
    text=re.sub(r"[$#@%&*!~?%{}()]"," ",text)
    
    return text

def printArticlesum(k):
    print("---------------------original sentence-----------------------")
    print("-------------------------------------------------------------")
    print(data["articles"][k])
    print("----------------------Summary sentence-----------------------")
    print("-------------------------------------------------------------")
    print(data["summaries"][k])
    return 0


def announcedone():
    duration=2000
    freq=440
    ws.Beep(freq,duration)

dict1={}
f1_score1=[]
precision1=[]
recall1=[]
f1_score2=[]
precision2=[]
recall2=[]
f1_scoreL=[]
precisionL=[]
recallL=[]
filenames1=load_data(datasets["text"],data_categories[0])
filenames2=load_data(datasets["summ"],data_categories[0])
print(filenames1)
print(filenames2)

"""----------load the data, sentences and summaries-----------"""
data["articles"]=[]
data["summaries"]=[]
k=0
while k<10:
 try:
    print(filenames1[k])
    data["articles"].append(parsetext(datasets["text"],data_categories[0],"%s"%filenames1[k]))
 except Exception as e:
    data["articles"].append("Could not read")
    print(e)
 try:
   print(filenames2[k])
   data["summaries"].append(parsetext(datasets["summ"],data_categories[0],"%s"%filenames2[k]))
 except Exception as e:
   data["summaries"].append("Could not read")
   print(e)
 
 reference=""
 reference=data["summaries"][k]
 print("number of words in reference summary",len(reference))
 s = ""
 for a in data['articles'][k]:
      s += a
 print("number of words in text file",len(s))
 sentences = sent_tokenize(s)
 print("number of sentences in text file",len(sentences))
 
 dict = {}
 text=""
 for a in sentences:
    temp = re.sub("[^a-zA-Z]"," ",a)
    temp = temp.lower()
    dict[temp] = a
    text+=temp
 stopwords = nltk.corpus.stopwords.words('english')
 word_frequencies = {}
 for word in nltk.word_tokenize(text):
    if word not in stopwords:
        if word not in word_frequencies.keys():
            word_frequencies[word] = 1
        else:
            word_frequencies[word] += 1
 max_freq = max(word_frequencies.values())

 for w in word_frequencies :
      word_frequencies[w]/=max_freq
 sentence_scores = {}
 for sent in sentences:
    for word in nltk.word_tokenize(sent.lower()):
        if word in word_frequencies.keys():
            if len(sent.split(' ')) < 30:
                if sent not in sentence_scores.keys():
                    sentence_scores[sent] = word_frequencies[word]
                else:
                    sentence_scores[sent] += word_frequencies[word]
 import heapq
 summary_sentences = heapq.nlargest(5, sentence_scores, key=sentence_scores.get)
 summary = ' '.join(summary_sentences)
 #print(summary)
 from rouge import Rouge
 print("number of words in the model summary",len(summary))
 rouge = Rouge()
 scores = rouge.get_scores(summary, reference)
 scores_rouge1 = [score['rouge-1']['r'] for score in scores]
 scores_rouge2 = [score['rouge-2']['r'] for score in scores]
 scores_rougeL = [score['rouge-l']['r'] for score in scores]                 
 recall1.append(scores_rouge1[0])
 recall2.append(scores_rouge2[0])
 recallL.append(scores_rougeL[0])
 scores_rouge1 = [score['rouge-1']['p'] for score in scores]
 scores_rouge2 = [score['rouge-2']['p'] for score in scores]
 scores_rougeL = [score['rouge-l']['p'] for score in scores]                 
 precision1.append(scores_rouge1[0])
 precision2.append(scores_rouge2[0])
 precisionL.append(scores_rougeL[0])
 scores_rouge1 = [score['rouge-1']['f'] for score in scores]
 scores_rouge2 = [score['rouge-2']['f'] for score in scores]
 scores_rougeL = [score['rouge-l']['f'] for score in scores]                 
 f1_score1.append(scores_rouge1[0])
 f1_score2.append(scores_rouge2[0])
 f1_scoreL.append(scores_rougeL[0])
 k=k+1

print("the average f1_score for rouge-1 is",math.fsum(f1_score1)/len(f1_score1))
print("the average precision for rouge-1 is",math.fsum(precision1)/len(precision1))
print("the average recall for rouge-1 is",math.fsum(recall1)/len(recall1))
print("the average f1_score for rouge-2 is",math.fsum(f1_score2)/len(f1_score2))
print("the average precision for rouge-2 is",math.fsum(precision2)/len(precision2))
print("the average recall for rouge-2 is",math.fsum(recall2)/len(recall2))
print("the average f1_score for rouge-L is",math.fsum(f1_scoreL)/len(f1_scoreL))
print("the average precision for rouge-L is",math.fsum(precisionL)/len(precisionL))
print("the average recall for rouge-L is",math.fsum(recallL)/len(recallL))

['102.txt', '104.txt', '105.txt', '106.txt', '108.txt', '109.txt', '110.txt', '111.txt', '118.txt', '128.txt', '129.txt', '130.txt', '131.txt', '132.txt', '148.txt', '149.txt', '150.txt', '151.txt', '152.txt', '153.txt', '155.txt', '156.txt', '159.txt', '160.txt', '161.txt', '163.txt', '168.txt', '169.txt', '17.txt', '170.txt', '173.txt', '174.txt', '181.txt', '19.txt', '205.txt', '206.txt', '214.txt', '225.txt', '227.txt', '228.txt', '229.txt', '230.txt', '64.txt', '65.txt', '66.txt', '67.txt', '72.txt', '73.txt', '92.txt', '93.txt']
['102.txt', '104.txt', '105.txt', '106.txt', '108.txt', '109.txt', '110.txt', '111.txt', '118.txt', '128.txt', '129.txt', '130.txt', '131.txt', '132.txt', '148.txt', '149.txt', '150.txt', '151.txt', '152.txt', '153.txt', '155.txt', '156.txt', '159.txt', '160.txt', '161.txt', '163.txt', '168.txt', '169.txt', '17.txt', '170.txt', '173.txt', '174.txt', '181.txt', '19.txt', '205.txt', '206.txt', '214.txt', '225.txt', '227.txt', '228.txt', '229.txt', '230.txt'

In [ ]:
print(f1_score1)
print(f1_score2)
print(f1_scoreL)

[0.1352169500607384, 0.16627078170070694, 0.1742243410705966, 0.17248459735199373, 0.09429657605616394, 0.1296829941462848, 0.15665796062826803, 0.12151067111233796, 0.2918149416299186, 0.21221864634479587]
[0.048533870096235665, 0.11190475977108846, 0.0789473658277055, 0.06790123234195336, 0.04874333399092451, 0.005780343860680121, 0.0366492118529168, 0.029605261044894977, 0.1146953355126478, 0.0741935452260147]
[0.1739130404834174, 0.1721518959846179, 0.16624684861321376, 0.21327967523288627, 0.09208103006949321, 0.10869564909321841, 0.12935323070320048, 0.11851851596851859, 0.28571428097009643, 0.177514789580722]
